# Data Processing Pipeline (Hoang's Method)

This notebook processes lung cancer data following Hoang's approach:
1. Load and encode data
2. Split by hospital group
3. Apply MICE imputation (avoiding data leakage)
4. Export processed datasets as CSV files

## Step 1: Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

## Step 2: Load Data

In [ ]:
# Load the dataset
df = pd.read_excel("data_ML_full.xlsx", header=0)

print("📋 Dataset shape:", df.shape)
print("\n🔍 First few rows:")
df.head()

## Step 3: Encode Patient IDs

In [ ]:
# Create simple numeric IDs
le_id = LabelEncoder()
df['ID_simple'] = le_id.fit_transform(df['ID_NO'].astype(str))

# Save ID mapping table for reference
id_mapping = pd.DataFrame({
    'Original_ID': le_id.classes_,
    'Simple_ID': range(len(le_id.classes_))
})
id_mapping.to_csv('ID_Mapping_Table.csv', index=False)

print("✅ ID encoding completed")
print(f"✅ ID mapping saved to 'ID_Mapping_Table.csv'")
print(f"\n🔍 Sample ID mapping:")
df[['ID_NO', 'ID_simple']].head()

## Step 4: Label Encode Categorical Columns

In [ ]:
# Identify non-numeric columns
non_num_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Exclude ID_NO, hosp_grp, and dead from encoding
exclude_cols = ["ID_NO", "hosp_grp", "dead"]
encode_cols = [c for c in non_num_cols if c not in exclude_cols]
encode_cols = list(set(encode_cols))

print(f"🧩 Non-numeric columns: {non_num_cols}")
print(f"\n✅ Columns to encode: {encode_cols}")

# Apply LabelEncoder to each column
df_encoded = df.copy()
for col in encode_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    print(f"🔢 Encoded '{col}' with {len(le.classes_)} unique values")

# Convert boolean columns to float
bool_cols = df_encoded.select_dtypes(include=['bool']).columns
if len(bool_cols) > 0:
    print(f"\n🧩 Converting {len(bool_cols)} boolean columns to float")
    df_encoded[bool_cols] = df_encoded[bool_cols].astype(float)

print(f"\n✅ Encoded dataframe shape: {df_encoded.shape}")
print(f"📊 Data types after encoding:")
print(df_encoded.dtypes.value_counts())

## Step 5: Identify Gene/Special Columns (to exclude from imputation)

In [ ]:
# Identify gene/mutation columns using pattern matching
keywords = ["pdl1", "ros1", "alk", "egfr", "exon21_all", "exon20_all", "exon19_all", "exon18_all", "ct", "cstage"]
pattern = r'(^|_)(' + '|'.join(keywords) + r')($|_)'

special_cols = [c for c in df_encoded.columns if re.search(pattern, c.lower())]
special_cols = [c for c in special_cols if c.lower() != "first_egfr"]

print("🧬 Gene/Cancer stage related columns (excluded from imputation):")
print(special_cols)
print(f"\n✅ Total special columns: {len(special_cols)}")

## Step 6: Split Data by Hospital Group (BEFORE Imputation)

**Note:** Train = Hospitals w+t (all data), Test = Hospital s (external validation)

In [ ]:
print("🏥 Splitting data by Hospital Group...")

# Training Set (Hospitals "w" and "t" - ALL data)
df_train = df_encoded[df_encoded["hosp_grp"].isin(["w", "t"])].copy()

# Test Set (Hospital "s" - external validation)
df_test = df_encoded[df_encoded["hosp_grp"] == "s"].copy()

# Separate features and target
X_train = df_train.drop(columns=["dead", "follow_up", "hosp_grp", "ID_NO"], errors="ignore")
y_train = df_train["dead"]

X_test = df_test.drop(columns=["dead", "follow_up", "hosp_grp", "ID_NO"], errors="ignore")
y_test = df_test["dead"]

print(f"✅ Split Summary:")
print(f"   - Train set (Hospitals w+t): {X_train.shape}")
print(f"   - Test set (Hospital s): {X_test.shape}")

## Step 7: Apply MICE Imputation (Avoiding Data Leakage)

In [ ]:
# Select numeric columns for imputation (excluding gene/special columns)
num_cols = X_train.select_dtypes(include=[np.number]).columns
cols_to_impute = [c for c in num_cols if c not in special_cols]

print(f"\n✅ Numerical columns selected for MICE: {len(cols_to_impute)}")
print(f"🧬 Excluded gene/stage columns: {len(special_cols)}")

# Initialize MICE Imputer
imputer = IterativeImputer(random_state=42, max_iter=1000, sample_posterior=True)

# Helper function to apply MICE imputation
def apply_mice(df_input, imputer_model, fit=False):
    """Apply MICE imputation to selected columns only"""
    df_out = df_input.copy()
    data_to_impute = df_out[cols_to_impute]
    
    if fit:
        # FIT on training data - learns relationships
        imputed_data = imputer_model.fit_transform(data_to_impute)
    else:
        # TRANSFORM on validation/test - uses learned relationships
        imputed_data = imputer_model.transform(data_to_impute)
    
    # Assign imputed data back to dataframe
    df_out[cols_to_impute] = imputed_data
    return df_out

print("\n🔄 Running MICE Imputation...")

# 1. Fit & Transform on TRAIN (hospitals w+t)
X_train_imputed = apply_mice(X_train, imputer, fit=True)
print("   - Train set imputed ✅")

# 2. Transform on TEST (hospital s) - no fit, avoids leakage
X_test_imputed = apply_mice(X_test, imputer, fit=False)
print("   - Test set imputed ✅")

# Verify no data leakage - check if gene columns unchanged
sample_gene = special_cols[0] if len(special_cols) > 0 else None
if sample_gene and sample_gene in X_test.columns:
    before = X_test[sample_gene].fillna(-999)
    after = X_test_imputed[sample_gene].fillna(-999)
    if (before == after).all():
        print(f"\n✅ Safety Check: Gene column '{sample_gene}' was NOT modified by MICE")
    else:
        print(f"\n⚠️ WARNING: Gene column '{sample_gene}' WAS modified!")

# Check remaining missing values
missing_remaining = X_train_imputed[cols_to_impute].isna().mean().sum()
print(f"\n📉 Missing values remaining in Train (imputed cols): {missing_remaining:.4f}")
print("\n🎉 Imputation Complete: No data leakage!")

## Step 8: Final Cleanup (Fill remaining NaN with -1)

In [ ]:
# Fill any remaining NaN values with -1
X_train_final = X_train_imputed.fillna(-1)
X_test_final = X_test_imputed.fillna(-1)

print("✅ Filled remaining NaN values with -1")
print(f"\n📊 Final dataset shapes:")
print(f"   - Train: {X_train_final.shape}")
print(f"   - Test: {X_test_final.shape}")

## Step 9: Export Processed Data to CSV Files

In [ ]:
# Combine features with target labels for export
train_data = X_train_final.copy()
train_data['dead'] = y_train.values

test_data = X_test_final.copy()
test_data['dead'] = y_test.values

# Export to CSV
train_data.to_csv('lung_train.csv', index=False)
test_data.to_csv('lung_test.csv', index=False)

print("✅ Data exported successfully!")
print(f"\n📁 Files created:")
print(f"   - lung_train.csv ({train_data.shape[0]} rows, {train_data.shape[1]} columns)")
print(f"   - lung_test.csv ({test_data.shape[0]} rows, {test_data.shape[1]} columns)")
print(f"   - ID_Mapping_Table.csv (for ID reference)")

print("\n🎯 Processing Complete!")

## Summary

This notebook processed the lung cancer dataset following Hoang's methodology:

**Key Steps:**
1. ✅ Loaded raw data from Excel
2. ✅ Encoded patient IDs (saved mapping table)
3. ✅ Label-encoded all categorical variables
4. ✅ Identified gene/mutation columns to preserve
5. ✅ Split data by hospital group (w+t for train, s for test)
6. ✅ Applied MICE imputation WITHOUT data leakage
7. ✅ Filled remaining NaN with -1
8. ✅ Exported 2 CSV files

**Output Files:**
- `lung_train.csv` - Training set (Hospitals w+t - ALL data)
- `lung_test.csv` - Test set (Hospital s - external validation)
- `ID_Mapping_Table.csv` - ID reference table

**Data Integrity:**
- ✅ No data leakage (imputer fitted only on training data from w+t)
- ✅ Gene/mutation columns preserved
- ✅ External test set (hospital s) remains independent